In [1]:
# Import the necessary packages and data
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
sns.set()
path = os.path.expanduser('~/Projects/capstone-two/data/raw/Harbor_Water_Quality.csv')
df = pd.read_csv(path, parse_dates=['Sample Date', 'Sample Time'])

C:\Users\Drew\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (5,6,10,11,12,13,20,21,26,27,35,38,40,51,53,59,60,61,64,65,67,68,76,82,83,88,94,95,97,98) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


There are dataype problems with certain columns, but before dealing with those, I first want to see what the dataframe looks like.

In [2]:
#df.shape

In [3]:
#df.head()

In [4]:
#df.info()

In [5]:
#df.describe()

In [6]:
#len(df['Sampling Location'].unique())

In [7]:
#df['Sampling Location'].unique()

There are 230 distinct locations, but I have a list of New York City Harbor Water Quality Survey Stations that is much shorter. I want to see how many samples I have from those sites. If I have a good amount, then I can continue without having to figure out the undefined locations. If I do need to use those locations, I could try utilizing their lat/long values and group them with the nearest site.

In [8]:
survey_stations = ['K1', 'K2', 'K3', 'K4', 'K5', 'K5A', 'K6',
                  'N1', 'N3B', 'N4', 'N5', 'N6', 'G2', 'N7', 'N8',
                  'N9', 'N16', 'NR1', 'E2', 'E4', 'E6', 'E7', 'E8',
                  'E10', 'E11', 'E12', 'E13', 'E14', 'E15', 'J1', 
                  'J2', 'J3', 'J5', 'J7', 'J8', 'J9A', 'J10', 'J11',
                  'J12', 'JA1', 'N9A', 'H3', 'J14', 'J16', 'AC1',
                  'AC1', 'AC2', 'BB2', 'BB4', 'BR1', 'BR3', 'BR5',
                  'CIC2', 'CIC3', 'F1', 'F5', 'FB1', 'FLC1', 'FLC2',
                  'GB1', 'GC3', 'GC4', 'GC5', 'GC6', 'HC1', 'HC2', 
                  'HC3', 'HR1', 'HR2', 'HR03', 'LN1', 'NC0', 'NC1',
                  'NC2', 'NC3', 'PB2', 'PB3', 'SP1', 'SP2', 'WC1',
                  'WC2', 'WC3'
                  ]
print(len(survey_stations))

82


In [9]:
df = df[df['Sampling Location'].isin(survey_stations)]
df = df.reset_index(drop=True)
len(df)

61967

From the dataset documentation, I am dropping columns from taken exclusively at the Oakwood Water Treatment Facility for now. I am also dropping all columns with less than 10,000 entries.

In [10]:
#df.info()

I don't want to look at data that is too old, so I am going to limit my dataset to readings from this millenium.

In [11]:
from datetime import datetime
df = df[df['Sample Date'] >= datetime(2000, 1, 1)]
df = df.reset_index(drop=True)

In [12]:
#df.info()

In [13]:
for col in df.columns:
    if 'Oakwood' in col:
        df = df.drop(columns=col)

#    elif 'Bottom' in col:
#        df = df.drop(columns=col)
    elif len(df[df[col].notnull()]) < 10000:
        df = df.drop(columns=col)
df = df.reset_index(drop=True)

In [14]:
#df.info()

Next I want to fix problematic columns

In [15]:
# Drop columns that I don't need
df = df.drop(['Current Direction (Current Direction)', 'Wind Direction (Wind Direction)',
              'Current Speed (knot)', 'Wind Speed (mph)', 'Sea State ', 'Type',
              'Enterococcus Top Sample Less Than or Greater Than Result'], axis=1)

In [16]:
# Fix 'Weather Condition' column
df['Weather Condition (Dry or Wet)'].unique()
df = df.replace(['Dry', 'Wet'], ['D', 'W'])

In [17]:
# Check which columns are numeric and create a list of object columns
obj_cols = []
ok_obj_cols = ['Sampling Location', 'Sample Date', 'Sample Time', 'Weather Condition (Dry or Wet)']
for col in df:
    if col not in ok_obj_cols:
        try:
            df[col] = pd.to_numeric(df[col])
        except:
            print(col)
            obj_cols.append(col)

Top Sample Photosysthetically Active Radiation Reference (400-700nm light (uE/S)
Top Sample Par (uE/S m2)
Secchi Depth (ft)
Bottom PH
Top Fecal Coliform Bacteria (Cells/100mL)
Top Enterococci Bacteria (Cells/100mL)
Top Nitrate/Nitrite (mg/L)
Top Ammonium (mg/L)
Top Ortho-Phosphorus (mg/L)
Top Silica (mg/L)
Total Phosphorus(mg/L)
Bottom Total Suspended Solid (mg/L)
Top Active Chlorophyll 'A' (µg/L)
Long
Lat


In [18]:
# Fix Bottom TSS
df['Bottom Total Suspended Solid (mg/L)'] = df['Bottom Total Suspended Solid (mg/L)'].replace('.', np.nan)

In [19]:
# Fix Secchi Disk
df['Secchi Depth (ft)'] = df['Secchi Depth (ft)'].replace('3..5', '3.5')

In [20]:
for i in df['Long']:
    try:
        pd.to_numeric(i)
    except:
        df['Long'] = df['Long'].replace(i, np.nan)

In [21]:
# Fix Bottom PH
df['Bottom PH'] = df['Bottom PH'].replace('N', np.nan)

In [22]:
# Fix Top Active Chlorophyl
df['Top Active Chlorophyll \'A\' (µg/L)'] = df['Top Active Chlorophyll \'A\' (µg/L)'].replace('.', np.nan)

In [23]:
# Preprocess Top Fecal Col
df['Top Fecal Coliform Bacteria (Cells/100mL)'] = df['Top Fecal Coliform Bacteria (Cells/100mL)'].replace('TNTC', np.nan)

In [24]:
# Fix bacteria
bacteria = ['Top Fecal Coliform Bacteria (Cells/100mL)',
           'Top Enterococci Bacteria (Cells/100mL)']
def fix_commas(string):
    '''Removes commas from strings representing numbers
    with values in the thousands'''
    try:
        string = pd.to_numeric(string)
    except:
        string = string.replace(',', '')
    return string

for col in bacteria:
    try:
        df[col] = df[col].apply(fix_commas)
        df[col] = pd.to_numeric(df[col])
    except:
        print(col)

In [25]:
# fix some of total phos
df['Total Phosphorus(mg/L)'] = df['Total Phosphorus(mg/L)'].replace(['0..454', '0,968'], ['0.454', '0.968'])

In [26]:
df['Top Ortho-Phosphorus (mg/L)'] = df['Top Ortho-Phosphorus (mg/L)'].replace('.', np.nan)

In [27]:
# fix < sign
less_than_cols = ['Top Nitrate/Nitrite (mg/L)', 'Top Ammonium (mg/L)',
                 'Top Ortho-Phosphorus (mg/L)', 'Top Silica (mg/L)',
                  'Total Phosphorus(mg/L)' 
                 ]
def drop_less(string):
    '''Removes commas from strings representing numbers
    with values in the thousands'''
    try:
        string = pd.to_numeric(string)
    except:
        string = pd.to_numeric(string.replace('<', ''))
    return string


for col in less_than_cols:
    try:
        df[col] = df[col].apply(drop_less)
        df[col] = pd.to_numeric(df[col])
    except:
        print(col)

In [28]:
def col_strip(string):
    '''Strips strings of trailing spaces and returns
    stripped string or numeric value'''
    if type(string) == str:
        string = string.replace(' ', '')
    return string

In [29]:
#df.info()

In [30]:
for i in df['Top Ortho-Phosphorus (mg/L)']:
    try:
        pd.to_numeric(i)
    except:
        print(i)

In [31]:
for i, lat in enumerate(df['Lat']):
    try:
        pd.to_numeric(lat)
    except:
        lat_long = lat.split(',')
        if len(lat_long) == 2:
            df.loc[:, 'Lat'][i] = lat_long[0]
            if len(lat_long[1]) > 1:
                df.loc[:, 'Long'][i] = lat_long[1]


C:\Users\Drew\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\Drew\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [32]:
for col in df:
    if col != 'Sample Date':
        try:
            df[col] = df[col].apply(col_strip)
            df[col] = pd.to_numeric(df[col])
        except:
            print(col)

Sampling Location
Sample Time
Weather Condition (Dry or Wet)


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31005 entries, 0 to 31004
Data columns (total 47 columns):
Sampling Location                                                                   31005 non-null object
Sample Date                                                                         31005 non-null datetime64[ns]
Sample Time                                                                         31005 non-null object
Weather Condition (Dry or Wet)                                                      30601 non-null object
Top Sample Temperature (ºC)                                                         18996 non-null float64
Bottom Sample Temperature (ºC)                                                      17733 non-null float64
Site Actual Depth (ft)                                                              27886 non-null float64
Top Sample Depth(ft)                                                                18939 non-null float64
Bottom Sample Depth (ft)       

In [34]:
df.describe()

,Top Sample Temperature (ºC),Bottom Sample Temperature (ºC),Site Actual Depth (ft),Top Sample Depth(ft),Bottom Sample Depth (ft),Top Salinity (psu),Bottom Salinity (psu),Top Conductivity (S/m),Bottom Conductivity (S/m),"CTD (conductivity, temperature, depth profiler) Top Dissolved Oxygen (mg/L)",...,Top Ortho-Phosphorus (mg/L),Top Total Kjeldhal Nitrogen (mg/L),Top Silica (mg/L),Total Phosphorus(mg/L),Top Total Suspended Solid (mg/L),Bottom Total Suspended Solid (mg/L),Top Active Chlorophyll 'A' (µg/L),Top Dissolved Organic Carbon (mg/L),Long,Lat
count,18996.000000,17733.000000,27886.000000,18939.000000,17836.000000,18968.000000,17720.000000,18758.000000,17528.000000,16217.000000,...,19297.000000,28533.000000,28507.000000,28554.000000,28503.000000,20709.000000,28474.000000,27732.000000,31003.000000,31003.000000
mean,18.703483,18.002141,28.630030,3.380010,32.244005,22.866737,24.809579,3.837574,4.090200,6.851332,...,0.165340,1.183278,2.151428,0.249150,14.555029,17.591278,16.219485,3.359917,-73.922439,40.683739
std,6.801709,6.523780,21.298745,0.749955,20.358388,5.269741,3.794550,4.396670,4.665498,2.871964,...,0.275925,1.865418,2.331622,0.330311,18.827639,21.909812,25.121986,1.290050,0.927987,0.926517
min,-1.740000,-1.870000,0.020000,0.700000,1.000000,0.120000,0.200000,0.020000,0.000000,-1.120000,...,0.000000,0.033000,0.014000,0.009000,0.000000,0.000000,0.090000,0.500000,-74.258833,-73.990053
25%,15.737500,15.000000,14.000000,3.000000,17.000000,21.160000,23.060000,2.730000,2.980000,5.080000,...,0.070000,0.608000,1.020000,0.129000,7.000000,7.400000,2.650000,2.700000,-73.989073,40.627000
50%,21.190000,20.340000,21.000000,3.000000,25.000000,24.060000,25.240000,3.370000,3.500000,6.410000,...,0.115000,0.858000,1.697000,0.181000,11.000000,12.000000,6.780000,3.200000,-73.899833,40.666667
75%,23.570000,22.830000,40.000000,4.000000,43.000000,26.220000,27.270000,3.820000,3.900000,8.080000,...,0.164000,1.270000,2.540000,0.258000,18.000000,21.000000,19.600000,3.800000,-73.847613,40.782500
max,30.010000,29.230000,230.000000,12.000000,107.000000,34.760000,44.730000,43.960000,62.560000,28.560000,...,5.210000,44.800000,28.834000,15.500000,1805.000000,704.400000,770.000000,97.000000,40.679429,40.914500


What would happen if I drop all rows that have fewer than 40 columns present?

In [ ]:
from collections import defaultdict
columns_represented = defaultdict(int)
for i, row in df.iterrows():
    columns_represented[row.notnull().sum()] +=1

In [39]:
drop_cols = []
for i, row in df.iterrows():
    if row.notnull().sum() < 23:
        drop_cols.append(i)
df1 = df.drop(drop_cols)

In [40]:
df1.shape

(20068, 47)

In [ ]:
for col in df:
    print(col, df[col].count())

I want to get an idea of the frequency each site is sampled.

In [ ]:
for site in survey_stations:
    site_df = df[df['Sampling Location'] == site]
    print(site, len(site_df), site_df['Sample Date'].min(), site_df['Sample Date'].max())